In [ ]:
# %% [markdown]
# # Task 2: Quantitative Analysis with TA-Lib and PyNance
# **Objective**: Calculate technical indicators (SMA, RSI, MACD) and financial metrics for stock price data.

# %% [markdown]
# ## 1. Setup
# Import libraries and configure settings

# %%


import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import talib
from pynance import get_data  # For advanced metrics

# Configure plotting
sns.set_style("darkgrid")
plt.rcParams["figure.figsize"] = (14, 6)

# %% [markdown]
# ## 2. Load Stock Data
# Assumes CSV files are in `data/` with columns: `Date,Open,High,Low,Close,Volume`

# %%
tickers = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA', 'TSLA']
stock_data = {}

for ticker in tickers:
    try:
        df = pd.read_csv(f"data/{ticker}_historical_data.csv", parse_dates=['Date'])
        df.set_index('Date', inplace=True)
        stock_data[ticker] = df
        print(f"✅ Loaded {ticker} ({len(df)} rows)")
    except FileNotFoundError:
        print(f"⚠️  File not found: data/{ticker}_historical_data.csv")

# Show sample data
stock_data['AAPL'].head()

# %% [markdown]
# ## 3. Calculate Technical Indicators
# Using TA-Lib for core calculations

# %%
for ticker, df in stock_data.items():
    # Simple Moving Average (20-day)
    df['SMA_20'] = talib.SMA(df['Close'], timeperiod=20)
    
    # Relative Strength Index (14-day)
    df['RSI_14'] = talib.RSI(df['Close'], timeperiod=14)
    
    # MACD (12, 26, 9)
    df['MACD'], df['MACD_Signal'], df['MACD_Hist'] = talib.MACD(
        df['Close'], 
        fastperiod=12, 
        slowperiod=26, 
        signalperiod=9
    )
    
    # Daily Returns and Volatility
    df['Daily_Return'] = df['Close'].pct_change()
    df['Volatility_20'] = df['Daily_Return'].rolling(window=20).std() * np.sqrt(252)

# Verify calculations
stock_data['TSLA'][['Close', 'SMA_20', 'RSI_14', 'MACD']].tail()

# %% [markdown]
# ## 4. Visualizations
# ### 4.1 Price and Moving Average

# %%
ticker = 'AAPL'  # Change to analyze different stocks
df = stock_data[ticker]

plt.figure(figsize=(14, 6))
plt.plot(df.index, df['Close'], label='Close Price', linewidth=2)
plt.plot(df.index, df['SMA_20'], label='20-Day SMA', linestyle='--')
plt.title(f"{ticker} - Price vs. 20-Day Moving Average", fontsize=16)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Price ($)", fontsize=12)
plt.legend()
plt.show()

# %% [markdown]
# ### 4.2 Relative Strength Index (RSI)

# %%
plt.figure(figsize=(14, 4))
plt.plot(df.index, df['RSI_14'], label='RSI (14)', color='purple', linewidth=2)
plt.axhline(70, linestyle='--', color='red', label='Overbought (70)')
plt.axhline(30, linestyle='--', color='green', label='Oversold (30)')
plt.title(f"{ticker} - Relative Strength Index", fontsize=16)
plt.ylim(0, 100)
plt.legend()
plt.show()

# %% [markdown]
# ### 4.3 MACD Indicator

# %%
plt.figure(figsize=(14, 5))
plt.plot(df.index, df['MACD'], label='MACD Line', color='blue')
plt.plot(df.index, df['MACD_Signal'], label='Signal Line', color='orange')
plt.bar(df.index, df['MACD_Hist'], label='Histogram', color='gray', alpha=0.3)
plt.title(f"{ticker} - MACD Indicator", fontsize=16)
plt.legend()
plt.show()

# %% [markdown]
# ### 4.4 Volatility Analysis

# %%
plt.figure(figsize=(14, 4))
plt.plot(df.index, df['Volatility_20'], color='crimson', linewidth=2)
plt.title(f"{ticker} - 20-Day Rolling Volatility (Annualized)", fontsize=16)
plt.ylabel("Volatility", fontsize=12)
plt.xlabel("Date", fontsize=12)
plt.show()

# %% [markdown]
# ## 5. Combined Analysis (Optional)
# Compare indicators for a specific date range

# %%
# Filter for last 90 days
recent_data = df.last("90D")

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Price and SMA
ax1.plot(recent_data.index, recent_data['Close'], label='Price')
ax1.plot(recent_data.index, recent_data['SMA_20'], label='20-Day SMA')
ax1.set_ylabel("Price ($)")
ax1.legend()

# RSI
ax2.plot(recent_data.index, recent_data['RSI_14'], label='RSI', color='purple')
ax2.axhline(70, color='red', linestyle='--')
ax2.axhline(30, color='green', linestyle='--')
ax2.set_ylabel("RSI")
ax2.set_ylim(0, 100)

plt.suptitle(f"{ticker} - Combined Technical Analysis (Last 90 Days)", fontsize=16)
plt.show()

# %% [markdown]
# ## 6. Export Results
# Save processed data for Task 3 (Correlation Analysis)

# %%
# Create processed data directory if it doesn't exist
os.makedirs("data/processed", exist_ok=True)

# Save each stock's data with indicators
for ticker, df in stock_data.items():
    df.to_csv(f"data/processed/{ticker}_with_indicators.csv")
    print(f"Saved: data/processed/{ticker}_with_indicators.csv")

# %% [markdown]
# ## Next Steps
# - Merge `task-2` branch into `main`
# - Proceed to **Task 3**: Correlation between news sentiment and these stock indicators

  Using cached ta_lib-0.6.3.tar.gz (376 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build ta-lib
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for ta-lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [58 lines of output]
      <string>:83: UserWarning: Cannot find ta-lib library, installation may fail.
      C:\Users\henok\AppData\Local\Temp\pip-build-env-81b7b6pv\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsWarning: `install_requires` overwritten in `pyproject.toml` (dependencies)
        corresp(dist, value, root_dir)
      C:\Users\henok\AppData\Local\Temp\pip-build-env-81b7b6pv\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:61: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License :: OSI Approved :: BSD License
      
              See

ModuleNotFoundError: No module named 'talib'